In [73]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from IPython.display import HTML
pd.set_option('display.max_rows', 500)

In [26]:
# Markes
print("welcome to rrr.lt")
page = requests.get("https://rrr.lt/")
soup = BeautifulSoup(page.content, 'html.parser')

mark_list = soup.find(class_ ="cars_list clearfix")
#mark_list = soup.find_all(class_ = "cars_list__links")
mark_list = mark_list.select(".cars_list__items a.cars_list__links")

mark_title ={ list(mark_list[sd].children)[0][:-1]:mark_list[sd]["href"] for sd in range(len(mark_list))}

markes = pd.DataFrame({
    "Markes": mark_title.items(),
})
markes
for key, value in mark_title.items():
    print(key)
pasirinkta_marke = input("Prasome pasirinkti marke.")


welcome to rrr.lt
AC
Acura
Aixam
Alfa Romeo
Alpine
ARO
Asia
Aston Martin
Audi
Bentley
BMW
Brilliance
Buick
Cadillac
Chery
Chevrolet
Chrysler
Citroen
Cupra
Dacia
Daewoo
DAF
Daihatsu
Dodge
Ferrari
Fiat
Ford
GAZ
Geo
GMC
Grecav-EKE
Honda
Hummer
Hymer
Hyundai
Infiniti
Isuzu
Iveco
Jaguar
Jeep
KIA
Lada
Lamborghini
Lancia
Land Rover
LDV
Lexus
Ligier
Lincoln
Lotus
Maserati
Mazda
McLaren
Mercedes-Benz
Mercury
MG
Microcar
Mini
Mitsubishi
Moskvich
Nissan
Oldsmobile
Opel
Peugeot
Piaggio
Plymouth
Pontiac
Porsche
Proton
Renault
Rolls-Royce
Rover
Saab
Saturn
Scion
Seat
Simca
Skoda
Smart
SsangYong
Subaru
Suzuki
Tata
Tazzari
Tesla
Toyota
UAZ
Vauxhall
VAZ
Volkswagen
Volvo
ZAZ
Visos markes
Prasome pasirinkti marke.Honda


In [27]:
# rrr.lt
# Modeliai
page = requests.get(mark_title[pasirinkta_marke])
soup = BeautifulSoup(page.content, 'html.parser')
models_list = soup.find(class_="models_hold")
#print(models_list.prettify())
#model_title = [sd.get_text() for sd in models_list.select(".models .model-title")]
models_list = models_list.select(".models .models__links")

model_title ={ models_list[sd]['data-label']:models_list[sd]["href"] for sd in range(len(models_list))}
for key, value in model_title.items():
    print(key)
pasirinkta_modelis = input("Prasome pasirinkti modeli.")

Accord (1977-1981)
Accord (1982-1985)
Accord (1986-1989)
Accord (1990-1993)
Accord (1994-1998)
Accord (1999-2002)
Accord (2003-2007)
Accord (2008-2016)
City (1981-1986)
City (1986-1994)
City (1996-2002)
City (2002-2008)
City (2008-)
Civic (1974-1976)
Civic (1977-1979)
Civic (1980-1981)
Civic (1982-1983)
Civic III (1983-1987)
Civic (1988-1991)
Civic (1992-1995)
Civic (1996-2000)
Civic (2001-2005)
Civic (2006-2011)
Civic (2011-2015)
Civic (2015-)
Clarity (2016-)
Concerto
CR-V (1995-2001)
CR-V (2002-2006)
CR-V (2007-2012)
CR-V (2013-2016)
CR-V (2017-)
CR-Z
CRX
CRX (1983-1987)
CRX (1987-1991)
E (2019-)
Element (2003-2011)
Fit (2008-2015)
Fit III (2013-)
FR-V
HR-V (1998-2003)
HR-V (2016-)
Insight
Integra
Jazz (1984-1986)
Jazz (2002-2008)
Jazz (2008-2014)
Jazz (2015-)
Legend (1986-1990)
Legend (1991-1996)
Legend III KA9 (1996-2006)
Legend (1997-1998)
Legend (2004-2008)
Legend V (2014-)
Logo
NSX
Odyssey
Pilot I (2002-2008)
Pilot II (2008-2015)
Prelude (1974-1982)
Prelude (1983-1984)
Prelude (

In [28]:
# detales kategorijos
page = requests.get(model_title[pasirinkta_modelis])
soup = BeautifulSoup(page.content, 'html.parser')

parts = soup.find(class_ = "parts")
parts = parts.find_all(class_ = "parts__text")
# parts_link = soup.find_all(class_ = "parts_all-categories")
parts_links = soup.select(".parts_all-categories a")
parts_title ={list(parts[sd].children)[0][:-1].replace('\n                    ', '')+" >>>>> "+str(list(parts[sd].children)[1].get_text()):parts_links[sd]["href"] for sd in range(len(parts))}

raktai = []
for key, value in parts_title.items():
    print(key)
    raktai.append(key)

 # Read Note below

pasirinkta_kategorija = input("Prasome pasirinkti daliu kategorija.")

r = re.compile(pasirinkta_kategorija)
pasirinkta_kategorija = list(filter(r.match, raktai))[0]
#pasirinkta_kategorija = re.search(pasirinkta_kategorija, parts_title.items())
#parts_list = parts.select("cars_list__items a.cars_list__links")

Apšvietimo sistema >>>>> (257)
Degalų mišinio sistema >>>>> (678)
Dujų išmetimo sistema >>>>> (97)
Durys >>>>> (715)
Galinė ašis >>>>> (248)
Galinės išorės detalės >>>>> (205)
Kėbulas/ kėbulo dalys/ kablys >>>>> (111)
Kitos detalės >>>>> (395)
Oro kondicionavimo- šildymo sistema/ radiatoriai >>>>> (581)
Pavarų dėžė/ sankaba/ transmisija >>>>> (220)
Priekinė ašis >>>>> (337)
Priekinės išorės detalės >>>>> (243)
Prietaisai/ jungikliai/ el. sistema >>>>> (1316)
Ratai/ padangos/ gaubtai >>>>> (22)
Salonas/ interjeras >>>>> (979)
Stabdžių sistema >>>>> (351)
Stiklai >>>>> (133)
Stiklų žibintų apiplovimo/ valymo sistema >>>>> (146)
Variklis >>>>> (690)
Prasome pasirinkti daliu kategorija.Stiklai


In [56]:
# Detales (full list)
pd.set_option('display.max_colwidth', None)

parts_spec = []
parts_detales = []
last = True
next_ = parts_title[pasirinkta_kategorija]
while(last):
    page = requests.get(next_)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    page_last = soup.find(class_ = "pages__items last") 
    page_last = page_last.find(class_ = "pages__links")
    
    if page_last["class"] == ["pages__links"]:
        next_ = page_last["href"]
    else:
        last = False
        
    parts_details =soup.select(".products__text p")
    parts_specs = soup.select(".products .products__image")
    parts_price = soup.select(".products .products__price")
    
    parts_spec += [parts_specs[sd]["title"] + " " + parts_specs[sd]["href"] + " " +  parts_price[sd].get_text() for sd in range(len(parts_specs))]
    
    parts_detales += [parts_details[sd].get_text() for sd in range(len(parts_details))]
    
a = 0
ordered_list = []
#for i in parts_spec:
    #ordered_list.append(str(a) + ") " + i)
    #a += 1
#ordered_list

item = []
links = []
kainos = []

url =  "(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
for i in range(len(parts_spec)):
    x = re.search(".+?(?=https:)", parts_spec[i])
    y = re.search(url, parts_spec[i])
    item.append(x.group())
    links.append(y.group())
    e = parts_spec[i].split()[-1]
    k = parts_spec[i].split()[-2]
    price = e + k
    kainos.append(price)
    
detales_info = [i for i in parts_detales if i.startswith(pasirinkta_marke)]

detales = pd.DataFrame({
    "Detalė": item,
    "Kaina": kainos, 
    "Info": detales_info,
    "Nuoroda": links,
})
detales

,Detalė,Kaina,Info,Nuoroda
0,Pagrindinis priekinių durų stiklas (keturdurio),€19.00,"Honda Accord 2007, 103kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/auv5144-m2h5as2-honda-accord-pagrindinis-priekiniu-duru-stiklas-keturdurio
1,Pagrindinis priekinių durų stiklas (keturdurio),€8.00,"Honda Accord 2003, 103kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/dnd26431-43r00035-honda-accord-pagrindinis-priekiniu-duru-stiklas-keturdurio
2,Pagrindinis galinių durų stiklas,€8.00,"Honda Accord 2003, 103kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/dnd26441-43r00122-honda-accord-pagrindinis-galiniu-duru-stiklas
3,"Mažasis ""A"" galinių durų stiklas",€8.00,"Honda Accord 2003, 103kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/dnd26479-43r00122-honda-accord-mazasis-a-galiniu-duru-stiklas
4,Pagrindinis galinių durų stiklas,€8.00,"Honda Accord 2003, 103kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/dnd26656-43r00122-honda-accord-pagrindinis-galiniu-duru-stiklas
5,"Mažasis ""A"" galinių durų stiklas",€7.00,"Honda Accord 2003, 103kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/dnd26669-43r00122-honda-accord-mazasis-a-galiniu-duru-stiklas
6,Pagrindinis galinių durų stiklas,€10.00,"Honda Accord 2004, 103kW, 2204cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/tim6908-honda-accord-pagrindinis-galiniu-duru-stiklas
7,Pagrindinis priekinių durų stiklas (keturdurio),€10.00,"Honda Accord 2004, 103kW, 2204cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/tim6907-honda-accord-pagrindinis-priekiniu-duru-stiklas-keturdurio
8,Pagrindinis galinių durų stiklas,€10.00,"Honda Accord 2004, 103kW, 2204cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/tim6906-honda-accord-pagrindinis-galiniu-duru-stiklas
9,Galinis stiklas,€20.09,"Honda Accord 2003, 103kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/aja2411-eom-honda-accord-galinis-stiklas


In [100]:
# SEARCH
a = input('Ka norite rasti?') # b 
detales[detales['Detalė'].str.contains(a)]

Ka norite rasti?šakė


,Detalė,Kaina,Info,Nuoroda
6,Priekinė šakė,€20.00,"Honda Accord 2009, 110kW, 2199cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/ara141623-tl0l-honda-accord-priekine-sake
7,Priekinė šakė,€20.00,"Honda Accord 2009, 110kW, 2199cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/ara141624-tl0r-honda-accord-priekine-sake
15,Priekinė šakė,€40.00,"Honda Accord 2009, 110kW, 2200cm3, Dyzelinas, Automatinė",https://rrr.lt/autodalis/dva24840-honda-accord-priekine-sake
16,Priekinė šakė,€40.00,"Honda Accord 2009, 110kW, 2200cm3, Dyzelinas, Automatinė",https://rrr.lt/autodalis/dva24839-honda-accord-priekine-sake
25,Priekinė šakė,€25.00,"Honda Accord 2011, 110kW, 2200cm3, Dyzelinas, Automatinė",https://rrr.lt/autodalis/fom114-honda-accord-priekine-sake
27,Priekinė šakė,€25.00,"Honda Accord 2009, 110kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/brz125959-honda-accord-priekine-sake
35,Priekinė šakė,€20.24,"Honda Accord 2008, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/mah2229-honda-accord-priekine-sake
36,Priekinė šakė,€20.24,"Honda Accord 2008, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/mah2228-honda-accord-priekine-sake
37,Priekinė šakė,€30.00,"Honda Accord 2009, 110kW, 2199cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/ara141628-332817-honda-accord-priekine-sake
38,Priekinė šakė,€30.00,"Honda Accord 2009, 110kW, 2199cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/ara141629-335217-honda-accord-priekine-sake


In [101]:
inp = int(input("Apie kuria preke norite daugiau sužinoti?"))

pasirinkta_preke = detales.loc[inp, 'Nuoroda']

page = requests.get(pasirinkta_preke)
soup = BeautifulSoup(page.content, 'html.parser')

detailed_info = soup.select(".product_desc .product_details")

detailed_test = [sd.get_text() for sd in soup.select(".product_desc .product_details")]
detailed_test = re.sub("\n", "|", detailed_test[1])
detailed_test = detailed_test.split("|")
del detailed_test[detailed_test == '']

k = 0
for i in range(len(detailed_test)):
    if k >= len(detailed_test)-1 :
        break
    print(detailed_test[k] + ": " + detailed_test[k+1])
    k += 2
    
print("Daugiau informacijos: " + detales.loc[inp, 'Nuoroda'])

Apie kuria preke norite daugiau sužinoti?49
Gamintojas: Honda
Serija: Accord
Modelis: Accord
Gamybos metai: 2008-2016
Metai: 2009
Kėbulo tipas: Sedanas
Vairo padėtis: Dešinėje
Varomieji ratai: Priekiniai
Kuro tipas: Dyzelinas
Variklio tūris, cm3: 2200
Variklio galia, kW: 110
Greičio dėžė: Mechaninė
Spalva: Pilka
Daugiau informacijos: https://rrr.lt/autodalis/val97633-honda-accord-priekine-sake


welcome to rrr.lt
AC
Acura
Aixam
Alfa Romeo
Alpine
ARO
Asia
Aston Martin
Audi
Bentley
BMW
Brilliance
Buick
Cadillac
Chery
Chevrolet
Chrysler
Citroen
Cupra
Dacia
Daewoo
DAF
Daihatsu
Dodge
Ferrari
Fiat
Ford
GAZ
Geo
GMC
Grecav-EKE
Honda
Hummer
Hymer
Hyundai
Infiniti
Isuzu
Iveco
Jaguar
Jeep
KIA
Lada
Lamborghini
Lancia
Land Rover
LDV
Lexus
Ligier
Lincoln
Lotus
Maserati
Mazda
McLaren
Mercedes-Benz
Mercury
MG
Microcar
Mini
Mitsubishi
Moskvich
Nissan
Oldsmobile
Opel
Peugeot
Piaggio
Plymouth
Pontiac
Porsche
Proton
Renault
Rolls-Royce
Rover
Saab
Saturn
Scion
Seat
Simca
Skoda
Smart
SsangYong
Subaru
Suzuki
Tata
Tazzari
Tesla
Toyota
UAZ
Vauxhall
VAZ
Volkswagen
Volvo
ZAZ
Visos markes
Prasome pasirinkti marke.Honda
Accord (1977-1981)
Accord (1982-1985)
Accord (1986-1989)
Accord (1990-1993)
Accord (1994-1998)
Accord (1999-2002)
Accord (2003-2007)
Accord (2008-2016)
City (1981-1986)
City (1986-1994)
City (1996-2002)
City (2002-2008)
City (2008-)
Civic (1974-1976)
Civic (1977-1979)
Civic (1980-1981)
Civ

['0) Pagrindinis priekinių durų stiklas (keturdurio) Kaina: 23.00 €Daugiau info >> https://rrr.lt/autodalis/imp718750-honda-element-pagrindinis-priekiniu-duru-stiklas-keturdurio',
 '1) Pagrindinis galinių durų stiklas Kaina: 23.00 €Daugiau info >> https://rrr.lt/autodalis/imp718611-honda-element-pagrindinis-galiniu-duru-stiklas',
 '2) Galinis šoninis kėbulo stiklas Kaina: 35.00 €Daugiau info >> https://rrr.lt/autodalis/imp717633-honda-element-galinis-soninis-kebulo-stiklas',
 '3) Galinis šoninis kėbulo stiklas Kaina: 35.00 €Daugiau info >> https://rrr.lt/autodalis/imp717631-honda-element-galinis-soninis-kebulo-stiklas']

In [67]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

import pandas as pd

root = tk.Tk()
root.geometry("500x500")
root.pack_propagate(False)
root.resizable(0, 0)

frame1 = tk.LabelFrame(root, text = "Detales")
frame1.place(height=250, width=500)

file_frame = tk.LabelFrame(root, text = "Ieskoti")
file_frame.place(height = 100, width = 400, rely=0.65,relx = 0)

##
tv1 = ttk.Treeview(frame1)
tv1.place(relheight=1,relwidth=1)

search = tk.Entry(file_frame)
search.place(rely=0.40, relx = 0.30)

treescrolly = tk.Scrollbar(frame1, orient="vertical", command=tv1.yview)
treescrollx = tk.Scrollbar(frame1, orient="horizontal", command=tv1.xview)
tv1.configure(xscrollcommand=treescrollx.set, yscrollcommand=treescrolly.set)
treescrollx.pack(side="bottom", fill = "x")
treescrolly.pack(side="right", fill= "y")

button1 = tk.Button(file_frame, text = "Visi prekės", command=lambda: Load_data())
button1.place(rely=0.35, relx=0.10)
button2 = tk.Button(file_frame, text = "Paieška", command=lambda: Search())
button2.place(rely=0.35, relx=0.65)

def Load_data():
    clear_data()
    tv1["column"] = list(detales.columns)
    tv1["show"] = "headings"
    for column in tv1["column"]:
        tv1.heading(column, text=column)
    
    df_rows = detales.to_numpy().tolist()
    for row in df_rows:
        tv1.insert("", "end", values=row)
    return None

def Search():
    clear_data()
    a = str(search.get())
    d1 = detales[detales['Detalė'].str.contains(a)]
        
    tv1["column"] = list(d1.columns)
    tv1["show"] = "headings"
    for column in tv1["column"]:
        tv1.heading(column, text=column)
    
    df_rows = d1.to_numpy().tolist()
    for row in df_rows:
        tv1.insert("", "end", values=row)
    return None

def clear_data():
    tv1.delete(*tv1.get_children())
root.mainloop()

In [46]:
detales = detales[detales['Detalė'].str.contains("Pagrindinis")]
detales

,Detalė,Kaina,Info,Nuoroda
0,Pagrindinis priekinių durų stiklas (keturdurio),€19.00,"Honda Accord 2007, 103kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/auv5144-m2h5as2-honda-accord-pagrindinis-priekiniu-duru-stiklas-keturdurio
1,Pagrindinis priekinių durų stiklas (keturdurio),€8.00,"Honda Accord 2003, 103kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/dnd26431-43r00035-honda-accord-pagrindinis-priekiniu-duru-stiklas-keturdurio
2,Pagrindinis galinių durų stiklas,€8.00,"Honda Accord 2003, 103kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/dnd26441-43r00122-honda-accord-pagrindinis-galiniu-duru-stiklas
4,Pagrindinis galinių durų stiklas,€8.00,"Honda Accord 2003, 103kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/dnd26656-43r00122-honda-accord-pagrindinis-galiniu-duru-stiklas
6,Pagrindinis galinių durų stiklas,€10.00,"Honda Accord 2004, 103kW, 2204cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/tim6908-honda-accord-pagrindinis-galiniu-duru-stiklas
7,Pagrindinis priekinių durų stiklas (keturdurio),€10.00,"Honda Accord 2004, 103kW, 2204cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/tim6907-honda-accord-pagrindinis-priekiniu-duru-stiklas-keturdurio
8,Pagrindinis galinių durų stiklas,€10.00,"Honda Accord 2004, 103kW, 2204cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/tim6906-honda-accord-pagrindinis-galiniu-duru-stiklas
12,Pagrindinis galinių durų stiklas,€3.89,"Honda Accord 2003, Nenurodyta, Mechaninė",https://rrr.lt/autodalis/kbi7299-honda-accord-pagrindinis-galiniu-duru-stiklas
18,Pagrindinis galinių durų stiklas,€9.00,"Honda Accord 2007, 103kW, 2200cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/auv2565-honda-accord-pagrindinis-galiniu-duru-stiklas
19,Pagrindinis priekinių durų stiklas (keturdurio),€10.00,"Honda Accord 2005, 96kW, 2000cm3, Dyzelinas, Mechaninė",https://rrr.lt/autodalis/rob17139-e643r00035-honda-accord-pagrindinis-priekiniu-duru-stiklas-keturdurio


In [74]:
import unittest
from selenium import webdriver
import page

class PythonOrgSearch(unittest.TestCase):
    def setUp(self):
        self.driver = webdriver.Firefox()
        self.driver.get("http://www.python.org")
    def test_search_in_python_org(self):

#Load the main page. In this case the home page of Python.org.
        main_page = page.MainPage(self.driver)
#Checks if the word "Python" is in title
        assert main_page.is_title_matches(), "python.org title doesn't match."
#Sets the text of search textbox to "pycon"
        main_page.search_text_element = "pycon"
        main_page.click_go_button()
        search_results_page = page.SearchResultsPage(self.driver)
#Verifies that the results page is not empty
        assert search_results_page.is_results_found(), "No results found."
def tearDown(self):
        self.driver.close()
if __name__ == "__main__":
        unittest.main()

ModuleNotFoundError: No module named 'page'